In [ ]:
from numpy.lib.polynomial import RankWarning
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
try:
  from pyspark.sql.types import *
  from pyspark.sql import SparkSession
  spark = SparkSession.builder.getOrCreate()
except Exception as e:
  !pip install pyspark
  from pyspark.sql.types import *
  from pyspark.sql import SparkSession
  spark = SparkSession.builder.getOrCreate()
from google.colab import drive
#mount google drive
drive.mount('/content/drive')
csvfilepath = '/content/drive/MyDrive/RP1data/SPSIRDATA.csv'
rawdf = pd.read_csv(csvfilepath)
display(rawdf)

#collect total slots
sparkdf = spark.createDataFrame(rawdf)
sparkdf.createOrReplaceTempView("slottable")
distinctslots = spark.sql("select distinct field1 from slottable")
allslots = [row.field1 for row in distinctslots.collect()]
#print(len(allslots))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=bcd7cd0227de21ec8ec0a47c0ad4fcfb39f86521e6f9b2e73a373333bf530cf7
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Mounted at /content/drive


,created_at,entry_id,field1,field2,field3
0,2020-01-01 00:00:00 UTC,1,IR20,1,2000-0-8T45:162:0
1,2020-01-01 08:35:00 UTC,2,IR40,0,2000-0-1T45:162:0
2,2020-01-02 02:12:00 UTC,3,IR22,1,2000-0-4T45:162:0
3,2020-01-02 17:45:00 UTC,4,IR12,0,2000-0-5T45:162:0
4,2020-01-03 05:48:00 UTC,5,IR18,1,2000-0-3T45:162:0
...,...,...,...,...,...
2764,2023-12-04 23:18:00 UTC,2765,IR39,1,2000-0-4T45:162:0
2765,2023-12-05 16:10:00 UTC,2766,IR27,1,2000-0-4T45:162:0
2766,2023-12-06 00:29:00 UTC,2767,IR24,1,2000-0-1T45:162:0
2767,2023-12-06 15:37:00 UTC,2768,IR22,1,2000-0-8T45:162:0


In [ ]:
#process raw dataframe
renamecolumns = {
    'created_at': 'Timestamp',
    'entry_id': 'entry_id',
    'field1': 'SlotID',
    'field2': 'Availability',
    'field3': 'field3'
}

df = rawdf.rename(columns=renamecolumns)
display(df)

,Timestamp,entry_id,SlotID,Availability,field3
0,2020-01-01 00:00:00 UTC,1,IR20,1,2000-0-8T45:162:0
1,2020-01-01 08:35:00 UTC,2,IR40,0,2000-0-1T45:162:0
2,2020-01-02 02:12:00 UTC,3,IR22,1,2000-0-4T45:162:0
3,2020-01-02 17:45:00 UTC,4,IR12,0,2000-0-5T45:162:0
4,2020-01-03 05:48:00 UTC,5,IR18,1,2000-0-3T45:162:0
...,...,...,...,...,...
2764,2023-12-04 23:18:00 UTC,2765,IR39,1,2000-0-4T45:162:0
2765,2023-12-05 16:10:00 UTC,2766,IR27,1,2000-0-4T45:162:0
2766,2023-12-06 00:29:00 UTC,2767,IR24,1,2000-0-1T45:162:0
2767,2023-12-06 15:37:00 UTC,2768,IR22,1,2000-0-8T45:162:0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

#extract features from timestamp
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Hourofday'] = df['Timestamp'].dt.hour
df['minute'] = df['Timestamp'].dt.minute
df['Dayofweek'] = df['Timestamp'].dt.dayofweek
df['Month'] = df['Timestamp'].dt.month
#print(df['Month'])

#encode slotid column
encoder = OneHotEncoder(sparse=False)
slotid_enc = encoder.fit_transform(df[['SlotID']])
cols_enc = encoder.get_feature_names_out(['SlotID'])
#create a df with all slotids
slotid_df = pd.DataFrame(slotid_enc, columns=cols_enc)
#display(slotid_df)

#extract features and target value
X = df[['minute', 'Hourofday', 'Dayofweek', 'Month']]
X = pd.concat([X, slotid_df], axis=1)
y = df['Availability']
print(X)


      minute  Hourofday  Dayofweek  Month  SlotID_IR1  SlotID_IR10  \
0          0          0          2      1         0.0          0.0   
1         35          8          2      1         0.0          0.0   
2         12          2          3      1         0.0          0.0   
3         45         17          3      1         0.0          0.0   
4         48          5          4      1         0.0          0.0   
...      ...        ...        ...    ...         ...          ...   
2764      18         23          0     12         0.0          0.0   
2765      10         16          1     12         0.0          0.0   
2766      29          0          2     12         0.0          0.0   
2767      37         15          2     12         0.0          0.0   
2768      35          6          3     12         0.0          0.0   

      SlotID_IR11  SlotID_IR12  SlotID_IR13  SlotID_IR14  ...  SlotID_IR46  \
0             0.0          0.0          0.0          0.0  ...          0.0   
1  

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
#split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#random forest classifier model
model = RandomForestClassifier(n_estimators=100, random_state = 42)
model.fit(X,y) #train model on whole dataset

RandomForestClassifier(random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score
#accuracy
y_pred = model.predict(X_test)
print(y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(f"model accuracy: ",accuracy)

[0 1 0 0 1 0 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0
 1 1 0 0 1 0 0 1 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1 0 1 1 1 0 1 1 0 0 0 1 0 1 0
 1 0 1 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 0 1 0 1 0 1 1 0 0 0 1 1
 1 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1 0 0 1 0 1 0 1 0 0 0 1 1 1 1
 0 1 1 1 0 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1 1 1 0 1 0 1 0 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 0 1 0 0 0 1 0 0
 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 0 1 1 0 1 1 0
 0 1 1 1 1 1 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 0 0 1 1 0 1 0 1
 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 0 0 1
 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0
 0 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0 1 1
 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1 1 1 1 0 1 0 0 1
 0 0 0 1 0 0 1 0 1 0 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 0 1 1 1 1 0 0 0 1 1 1 0
 1 1 1 1 1 1 1 0 0 1 1 0 

In [ ]:
#input datetime
print("Predict slot availability at time: ")
giventime = input("MM dd HH mm = ")
giventime = pd.to_datetime(giventime, format = '%m %d %H %M')
givenday = giventime.dayofweek
givenhour = giventime.hour
givenminute = giventime.minute
givenmonth = giventime.month

#predict available slots for the given time of day
X_given = pd.DataFrame({
    'minute':[givenminute],
    'Hourofday':[givenhour],
    'Dayofweek':[givenday],
    'Month':[givenmonth]
})
X_given = pd.concat([X_given, slotid_df], axis=1)
X_given.fillna(0, inplace=True)
#print(X_given)

slot_prediction = model.predict(X_given)
print(slot_prediction)
available_slots = df.loc[slot_prediction == 1, 'SlotID'].unique()
print("Predicted available slots: ",available_slots)
print(len(available_slots))

#probability of slots to be available
probability = model.predict_proba(X_given)
available_index = [index for index, pred in enumerate(slot_prediction) if pred == 1]
available_probability = probability[available_index][:,1]
slots_probability = dict(zip(available_slots, available_probability))
print("Predicted probabilities: ",slots_probability)
print(len(slots_probability))

Predict slot availability at time: 
MM dd HH mm = 11 11 11 11
[0 0 1 ... 0 1 1]
Predicted available slots:  ['IR22' 'IR12' 'IR18' 'IR7' 'IR29' 'IR20' 'IR14' 'IR33' 'IR46' 'IR5'
 'IR45' 'IR16' 'IR4' 'IR50' 'IR28' 'IR3' 'IR25' 'IR15' 'IR27' 'IR1' 'IR19'
 'IR10' 'IR21' 'IR48' 'IR36' 'IR17' 'IR13' 'IR23' 'IR2' 'IR47' 'IR26'
 'IR30' 'IR35' 'IR38' 'IR11']
35
Predicted probabilities:  {'IR22': 0.56, 'IR12': 0.54, 'IR18': 0.69, 'IR7': 0.6, 'IR29': 0.61, 'IR20': 0.59, 'IR14': 0.6, 'IR33': 0.6, 'IR46': 0.6, 'IR5': 0.56, 'IR45': 0.6, 'IR16': 0.62, 'IR4': 0.55, 'IR50': 0.59, 'IR28': 0.65, 'IR3': 0.63, 'IR25': 0.56, 'IR15': 0.57, 'IR27': 0.56, 'IR1': 0.56, 'IR19': 0.56, 'IR10': 0.59, 'IR21': 0.6, 'IR48': 0.63, 'IR36': 0.63, 'IR17': 0.66, 'IR13': 0.61, 'IR23': 0.6, 'IR2': 0.58, 'IR47': 0.64, 'IR26': 0.54, 'IR30': 0.64, 'IR35': 0.58, 'IR38': 0.6, 'IR11': 0.61}
35
